In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import dask.dataframe as dd
from dask.multiprocessing import get

# !pip install swifter --user
import swifter
from pandas.io.json import json_normalize

from joblib import Parallel, delayed
from multiprocessing import cpu_count

import threading

cores = cpu_count() #Number of CPU cores on your system
fracciones = cores * 2

C:\Users\matias\AppData\Roaming\Python\Python36\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
def getTimeResponse(row):
    proxiesTemp = {
        "http": row['IP Address']
    }
    responseA = requests.post('https://www.twitter.com', headers=headersChrome, proxies=proxiesTemp)
    timeA = responseA.elapsed.total_seconds()
    print(timeA)
    return timeA
proxiesDf.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,190.184.144.82,52443,CO,Colombia,elite proxy,no,yes,5 seconds ago
1,113.160.234.147,54329,VN,Vietnam,elite proxy,no,yes,5 seconds ago
2,45.64.179.99,53281,IN,India,elite proxy,no,yes,5 seconds ago
3,170.84.93.106,42276,BR,Brazil,elite proxy,no,yes,5 seconds ago
4,3.212.104.192,80,US,United States,elite proxy,no,yes,17 seconds ago


In [3]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

In [4]:
proxiesWithTime = proxiesDf.copy()
proxiesWithTime.loc[:, 'timeResponse'] = np.NaN

def parallCheckResponses(arg):
    cuenta_errores = 0
    proxiesWithTime.loc[arg.index.values, 'timeResponse'] = arg.apply(getTimeResponse, axis=1)
    return False

Parallel(
    n_jobs=fracciones, backend="threading",
)(map(
    delayed(parallCheckResponses),
    chunkIt(proxiesWithTime, fracciones)
))
proxiesWithTime

0.692389
0.708872
0.694476
0.706438
0.702509
0.7627650.7685810.758083
0.720673

0.7056940.7576980.7745070.714798
0.752617


0.7256090.706445
0.7717270.750123
0.786648




0.7766910.7843840.78956
0.796306


0.796744
0.682623
0.760783
0.712773
0.753621
0.707284
0.721311
0.724105
0.786574
0.73093
0.724443
0.771224
0.755304
0.739066
0.712393
0.69642
0.763993
0.766777
0.713474
0.717299
0.793218
0.7717580.793349

0.713843
0.707882
0.829396
0.75782
0.713822
0.746863
0.72254
0.773404
0.702735
0.737361
0.712278
0.734787
0.739991
0.7342590.703379

0.775763
0.753106
0.77957
0.793113
0.809206
0.743515
0.8434870.776739

0.855806
0.707923
0.718107
0.724511
0.701797
0.696994
0.705373
0.695375
0.710667
0.708416
0.768728
0.750321
0.717483
0.69222
0.706592
0.781754
0.705592
0.7510770.781007

0.747495
0.762856
0.742267
0.770952
0.792026
1.167221
0.710659
0.702669
0.783549
0.780297
3.777722
1.32921


,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,timeResponse
0,190.184.144.82,52443,CO,Colombia,elite proxy,no,yes,5 seconds ago,0.768581
1,113.160.234.147,54329,VN,Vietnam,elite proxy,no,yes,5 seconds ago,0.730930
2,45.64.179.99,53281,IN,India,elite proxy,no,yes,5 seconds ago,0.855806
3,170.84.93.106,42276,BR,Brazil,elite proxy,no,yes,5 seconds ago,0.770952
4,3.212.104.192,80,US,United States,elite proxy,no,yes,17 seconds ago,0.762765
5,138.68.57.96,8080,US,United States,anonymous,no,no,17 seconds ago,0.712773
6,167.99.11.221,8080,US,United States,anonymous,no,no,17 seconds ago,0.773404
7,51.140.29.241,3128,GB,United Kingdom,elite proxy,no,no,17 seconds ago,0.768728
8,54.37.21.29,3128,FR,France,elite proxy,no,yes,17 seconds ago,0.706438
9,104.248.75.171,8080,US,United States,anonymous,no,no,17 seconds ago,0.713843


In [5]:
bestProxies = proxiesWithTime.sort_values(by='timeResponse')
bestProxies = bestProxies[:50]
def getProxie():
    return {
        "http": bestProxies.sample(1)['IP Address'].values[0]
    }
bestProxies.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,timeResponse
46,115.127.114.76,39003,BD,Bangladesh,elite proxy,no,yes,4 minutes ago,0.682623
19,177.130.59.121,36354,BR,Brazil,elite proxy,no,no,3 minutes ago,0.692220
45,171.255.192.118,8080,VN,Vietnam,anonymous,no,yes,4 minutes ago,0.692389
20,89.24.124.82,44504,CZ,Czech Republic,elite proxy,no,no,3 minutes ago,0.694476
94,80.65.28.57,30962,RU,Russian Federation,elite proxy,no,yes,19 minutes ago,0.695375


In [6]:
dataTweets = pd.read_pickle('../datasets/TweetsDataset.pkl')
scrapedUsers = pd.read_pickle('../datasets/TweetsScrapedUsers.pkl')
toScrap = pd.read_pickle('../datasets/UsersDataset.pkl')

print(len(toScrap), 'usuarios totales')
print(len(scrapedUsers), 'usuarios escrapeados')

toScrap.index = toScrap.index.map(int)
toScrap = toScrap.loc[toScrap.index.difference(scrapedUsers.index.map(int)).values]

print(len(toScrap), 'usuarios restantes')
print(len(toScrap) * 15 * 20, 'tweets restantes')
print(len(toScrap) * 15, 'paginas restantes')
toScrap.head(2)

15482 usuarios totales
1444 usuarios escrapeados
14038 usuarios restantes
4211400 tweets restantes
210570 paginas restantes


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
6434002,6434002,arielblanco,Ariel Blanco,4269,CEO de Trinahost SRL,Argentina,False,trinahost.com,1,2007-05-01 00:00:00,2019-07-01 00:26:02.693518,616,15729,32,False
7697752,7697752,melquiadess,josé,58551,médico. overachiever.,"Buenos Aires, Argentina.",13 de septiembre,instagram.com/melquiadess,0,2007-07-01 00:00:00,2019-06-30 00:01:18.817729,41685,9674,407,False


In [ ]:
# dataTweets = pd.DataFrame(columns=[
#     'tweet_id',
#     'user_id',
#     'screen_name',
#     'complete_name',
#     'text',
#     'hashtags',
#     'mencioned_user_id',
#     'mencioned_screen_name',
#     'is_reply',
#     'reply_screen_name',
#     'reply_user_id',
#     'all_reply_screen_name',
#     'all_reply_user_id',
#     'is_retweet',
#     'retweet_user_id',
#     'retweet_screen_name',
#     'retweet_complete_name',
#     'retweet_count',
#     'reply_count',
#     'favorite_count',
#     'num_hashtags',
#     'num_urls',
#     'num_mentions',
#     'created_at',
#     'crawled_at'
# ])

lock = threading.RLock()

def getTweetData(tweet):
    text = tweet.findAll(attrs={'class':'content'})[0].findAll(attrs={'class':'tweet-text'})[0].text
    hashtagsSrc = tweet.findAll(attrs={'class':'content'})[0].findAll(attrs={'class':'tweet-text'})[0].\
        findAll(attrs={'class':'twitter-hashtag'})
    hashtags = []
    if(len(hashtagsSrc)>0):
        hashtags = [x.text[1:] for x in hashtagsSrc]
    amountHashtags = len(hashtags)
    retweetData = tweet.findAll(attrs={'class':'js-retweet-text'})
    isRetweeted = False
    retweet_userid = np.NaN
    retweet_displayName = np.NaN
    retweet_username = np.NaN
    
    username = False
    userid = False
    displayName = False
    if(len(retweetData) > 0):
        isRetweeted = True
        retweetUser = retweetData[0].findAll(attrs={'class':'js-user-profile-link'})[0]
        
        userid = retweetUser.attrs['data-user-id']
        displayName = retweetUser.findAll('b')[0].text
        username = retweetUser.attrs['href'][1:]
    else:
        username = tweet.findAll(attrs={'class':'username'})[0].text[1:]
        userid = tweet.findAll(attrs={'class': 'js-action-profile'})[0].attrs['data-user-id']
        displayName = tweet.findAll(attrs={'class':'js-stream-tweet'})[0].attrs['data-name']
    repply_data = tweet.findAll(attrs={'class':'ReplyingToContextBelowAuthor'})
    is_reply = False
    all_reply_screen_name = []
    all_reply_user_id = []
    if(len(repply_data)==1):
        repply_data = repply_data[0]
        is_reply = True
        for repplied_user in repply_data.findAll('a'):
            all_reply_screen_name.append(repplied_user.attrs['href'][1:])
            all_reply_user_id.append(repplied_user.attrs['data-user-id'])
    reply_count =  tweet.findAll(attrs={'class':'ProfileTweet-action--reply'})[0]\
    .findAll(attrs={'class':'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
    retweet_count = tweet.findAll(attrs={'class':'ProfileTweet-action--retweet'})[0]\
    .findAll(attrs={'class':'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
    favorite_count = tweet.findAll(attrs={'class':'ProfileTweet-action--favorite'})[0]\
    .findAll(attrs={'class':'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
    tweetContainer = tweet.findAll(attrs={'class':'js-tweet-text-container'})[0]
    mencioned_user_id = []
    mencioned_screen_name = []
    for mencioned in tweetContainer.findAll('a', {'data-mentioned-user-id': re.compile(r'.*')},):
        mencioned_user_id.append(mencioned.attrs['data-mentioned-user-id'])
        mencioned_screen_name.append(mencioned.attrs['href'][1:])
    links_count = len(tweetContainer.findAll(attrs={'class':'twitter-timeline-link'}))
    with lock:
        global dataTweets
        dataTweets.loc[int(tweet.attrs['data-item-id'])] = {
            'tweet_id': int(tweet.attrs['data-item-id']),
            'user_id': userid,
            'screen_name': username,
            'complete_name': displayName,
            'text': text,
            'hashtags': hashtags,

            'mencioned_user_id': mencioned_user_id,
            'mencioned_screen_name':mencioned_screen_name,

            'is_reply': is_reply,
            'reply_screen_name': np.NaN if len(all_reply_screen_name)==0 else all_reply_screen_name[0],
            'reply_user_id': np.NaN if len(all_reply_user_id)==0 else all_reply_user_id[0],
            'all_reply_screen_name': all_reply_screen_name,
            'all_reply_user_id': all_reply_user_id,

            'is_retweet': isRetweeted,
            'retweet_user_id': retweet_userid,
            'retweet_screen_name': retweet_username,
            'retweet_complete_name': retweet_displayName,

            'retweet_count': retweet_count,
            'reply_count': reply_count,
            'favorite_count': favorite_count,
            'num_hashtags': amountHashtags,
            'num_urls': links_count,
            'num_mentions': len(mencioned_screen_name),

            'created_at': pd.Timestamp(int(tweet.findAll(attrs={'class':'js-short-timestamp'})[0].attrs['data-time']), unit='s'),
            'crawled_at': pd.Timestamp.now()
        }

    
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getTweetsOfUser(userId, userName):
    tweetUserCount = 0
    pageResponse = requests.get('https://twitter.com/'+userName, headers=headersChrome, proxies=getProxie())
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', recursive=False)
    tweetUserCount += len(tweetList)
    if(len(tweetList) == 0):
        print('NO TWEETS', userName)
        return False
    for tweet in tweetList:
        getTweetData(tweet)
    urlMore = 'https://twitter.com/i/profiles/show/'+userName+'/timeline/tweets'
    maxPages = 9
    lastMaxKey = dataMax
    for pageNum in range(maxPages):
        if(pageNum % 2 == 0):
            print('PAG '+str(pageNum)+' de 9.', userName)
        moreResponseRaw = requests.get(
            urlMore,
            headers=headersMore,
            proxies=getProxie(),
            params= {
                'max_position': lastMaxKey,
                'reset_error_state': False,
                'include_available_features': 1,
            }
        )
        moreResponse = json.loads(moreResponseRaw.text)
        moreTweets = BeautifulSoup(moreResponse['items_html'], 'html.parser')
        tweetList = moreTweets.findAll('li', recursive=False)
        tweetUserCount += len(tweetList)
        if(len(tweetList) == 0):
            break
        for tweet in tweetList:
            getTweetData(tweet)
        lastMaxKey = moreResponse['min_position']
    with lock:
        global scrapedUsers
        scrapedUsers.loc[userId] = [userId, userName, tweetUserCount]

In [ ]:
tweetsErrors = 0
tweetsErrUsers = []

def saveResults():
    with lock:
        print('--CHECKPOINT--')
        global scrapedUsers
        global dataTweets
        scrapedUsers.to_pickle('../datasets/TweetsScrapedUsers.pkl')
        dataTweets.to_pickle('../datasets/TweetsDataset.pkl')

def parallTweets(args):
    usersTotal = len(args)
    currUser = 1
    for userId, userName in zip(args.id, args.screen_name):
        print('--------------'+str(currUser)+' de '+str(usersTotal)+'--------------')
        if(currUser % 2 == 0):
            # Descargar a un archivo cada 2 usuarios
            saveResults()
        currUser += 1
        try:
            getTweetsOfUser(userId, userName)
        except KeyboardInterrupt:
            raise
        except:
            global tweetsErrors
            global tweetsErrUsers
            print('ERR', userName)
            tweetsErrors += 1
            tweetsErrUsers.append(userName)
    saveResults()

try:
    Parallel(
        n_jobs=fracciones, backend="threading",
    )(map(
        delayed(parallTweets),
        chunkIt(toScrap[['id', 'screen_name']], cores * 2)
    ))
except KeyboardInterrupt:
    raise

print('DONE')
print('ERRORS:',tweetsErrors)
print(tweetsErrUsers)

In [ ]:
print(len(scrapedUsers))
display(scrapedUsers.head())
print(len(dataTweets))
display(dataTweets.head())
saveResults()